# The Iris Dataset with Graph Execution



## Setup program

### Configure imports



In [0]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

## Import and parse the training dataset


### Download the dataset



In [0]:
training_file_url = "http://download.tensorflow.org/data/iris_training.csv"
training_file_path = tf.keras.utils.get_file(
    fname=os.path.basename(training_file_url),
    origin=training_file_url)

print("Location of the training data file: {}".format(training_file_path))

### Inspect the data



In [0]:
!head -n5 {training_file_path}

### Parse the dataset


In [0]:
csv_column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
y_name = 'species'

training_dataframe = pd.read_csv(training_file_path, names=csv_column_names, header=0)
training_features, training_target = training_dataframe, training_dataframe.pop(y_name)

### Create the training tf.data.Dataset



In [0]:
feature_columns = []
for key in training_features.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

## Select the type of model



### Create a Classifier



In [0]:
classifier = tf.estimator.DNNClassifier(
  feature_columns=feature_columns,
  hidden_units=[10, 10],
  n_classes=3)

## Train the model



### Define the loss and gradient function



In [0]:
def train_input_fn(features, labels, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  dataset = dataset.shuffle(1000).repeat().batch(batch_size)
  return dataset

### Create an optimizer



### Training loop



In [0]:
batch_size = 100
steps = 1000

classifier.train(
    input_fn=lambda:train_input_fn(training_features, training_target, batch_size),
    steps=steps)

### Visualize the loss function over time

## Evaluate the model's effectiveness



### Setup the test dataset



In [0]:
test_file_url = "http://download.tensorflow.org/data/iris_test.csv"
test_file_path = tf.keras.utils.get_file(
    fname=os.path.basename(test_file_url),
    origin=test_file_url)

print("Location of the test data file: {}".format(test_file_path))

test_dataframe = pd.read_csv(test_file_path, names=csv_column_names, header=0)
test_features, test_target = test_dataframe, test_dataframe.pop(y_name)


### Evaluate the model on the test dataset



In [0]:
def eval_input_fn(features, labels, batch_size):

  features=dict(features)
    
  if labels is None:
    inputs = features
  else:
    inputs = (features, labels)

  dataset = tf.data.Dataset.from_tensor_slices(inputs)

  assert batch_size is not None, "batch_size must not be None"
  dataset = dataset.batch(batch_size)

  return dataset

In [0]:
eval_result = classifier.evaluate(
  input_fn=lambda:eval_input_fn(test_features, test_target, batch_size))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

## Use the trained model to make predictions


In [0]:
class_names = ['Iris Setosa', 'Iris Versicolor', 'Iris Virginica']
 
predict_features = {
  'sepal_length': [5.1, 5.9, 6.9],
  'sepal_width': [3.3, 3.0, 3.1],
  'petal_length': [1.7, 4.2, 5.4],
  'petal_width': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
  input_fn=lambda:eval_input_fn(predict_features, labels=None, batch_size=batch_size))

for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  class_name = class_names[class_id]
  probability = pred_dict['probabilities'][class_id]

  print("Prediction {} with probability {:.1f}%".format(class_name, 100 * probability))
